# Notebook Lapage / Tanguy Mathieu / P6 Data Analyst 10/2022

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chi2_contingency as chi2_contingency
from scipy.stats import f_oneway
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import bartlett
from scipy.stats import shapiro
from scipy.stats import kstest
import datetime
from datetime import date
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 1ère PARTIE: CHARGEMENT,NETTOYAGE, PREPARATION DES DONNEES

### chargement du dossier transaction et premières vérifications puis nettoyage des données

In [ ]:
transaction = pd.read_csv('transactions.csv')
transaction.head()

In [ ]:
transaction.describe(include='all')

In [ ]:
transaction.info()

### recherche des doublons et leur suppression

In [ ]:
duplicate_transaction = transaction[transaction.duplicated()]
duplicate_transaction

In [ ]:
transaction1 = transaction.drop_duplicates()
transaction1

In [ ]:
duplicate_transaction1 = transaction1[transaction1.duplicated()]
duplicate_transaction1

In [ ]:
transaction1.describe()

### recherche et suppression des valeurs 's_0'

In [ ]:
s0 = transaction1.loc[transaction1['session_id']=='s_0']
s0

In [ ]:
transaction2 = transaction1.drop(transaction1.loc[transaction1['session_id']=='s_0'].index)
transaction2

In [ ]:
transaction2.isnull().sum()

In [ ]:
transaction2.isna().sum()

### changement du format objet en date pour la colonne 'date'

In [ ]:
transaction2['date']= pd.to_datetime(transaction2.date).dt.strftime('%Y-%m-%d')
transaction2['date']= pd.to_datetime(transaction2['date'])

### chargement du dossier products et premières vérifications puis nettoyage des données

In [ ]:
product = pd.read_csv('products.csv')
product.head()

In [ ]:
product.describe(include='all')

### recherche des doublons dans le dossier 'product'

In [ ]:
product.info()

In [ ]:
duplicate_product = product[product.duplicated()]
duplicate_product

### recherche et suppression des valeurs inférieur à 0 dans les prix

In [ ]:
product.loc[product['price']<0]

In [ ]:
product1 = product.drop(product.loc[product['price']<0].index)
product1

In [ ]:
t0 = product1.loc[product1['id_prod']=='T_0']
t0

In [ ]:
product1.describe(include='all')

In [ ]:
product1.isnull().sum()

In [ ]:
product1.isna().sum()

## groupage du dossier transaction et product

In [ ]:
transac_prod = pd.merge(transaction2,product1,how='left', on='id_prod')
transac_prod

In [ ]:
transac_prod.info()

In [ ]:
transac_prod.isnull().sum()

In [ ]:
transac_prod.describe()

In [ ]:
group_categ= round(transac_prod.groupby('categ')['price'].mean(),2)
group_categ

In [ ]:
group_categ.iloc[0]

In [ ]:
df_categ0 = transac_prod.loc[transac_prod['categ']==0.0]
df_categ0.describe()

In [ ]:
transac_prod.loc[transac_prod['categ'].isna()]


### remplacement des valeurs NaN dans 'categ' par 0.0

In [ ]:
transac_prod['categ'].fillna(value=0.0, inplace=True)

### imputation par la moyenne pour les prix affichant NaN dans la categorie 0.0

In [ ]:
transac_prod['price'].fillna(value=group_categ.iloc[0],inplace=True)

In [ ]:
transac_prod.info()

In [ ]:
transac_prod.info()

### chargement du dossier customers 

In [ ]:
customer = pd.read_csv('customers.csv')
customer.head()

In [ ]:
customer.describe(include='all')

In [ ]:
customer.info()

In [ ]:
duplicate_customer = customer[customer.duplicated()]
duplicate_customer

In [ ]:
customer.isnull().sum()

In [ ]:
customer.isna().sum()

In [ ]:
customer.sort_values(["client_id"], axis=0, ascending=False, inplace=True)
customer.head()

### découverte de 2 clients témoin 'ct_0' et 'ct_1' que je supprime

In [ ]:
customer =customer.drop(customer.loc[customer['client_id']=='ct_0'].index)
customer =customer.drop(customer.loc[customer['client_id']=='ct_1'].index)
customer.sort_values(["client_id"], axis=0, ascending=False, inplace=True)
customer.head()

### création de la colonne age

In [ ]:
customer['age'] = 2023-customer['birth']
customer.head()

### vérification par describe si pas d'age abérants

In [ ]:
customer.describe()

### création d'une colonne de découpage par tranche d'age

In [ ]:
customer['tranche age'] = pd.cut(x=customer['age'], bins=[1,20,30,40,50,60,70,80,100],
                    labels=['1 à 20','21 à 30','31 à 40','41 à 50','51 à 60',
                            '61 à 70','71 à 80' ,'81 à 100'],include_lowest=True)
customer

## groupage des 2 premiers dossier avec le dossier 'customer'

In [ ]:
df_total = pd.merge(transac_prod,customer,how='left',on='client_id')
df_total

In [ ]:
df_total.isnull().sum()

In [ ]:
df_total.isna().sum()

### recherche si des clients ont des transactions anormalement élevées

In [ ]:
pro = df_total['client_id'].value_counts().reset_index().head(10)
pro.sort_values(by='client_id',ascending=False).head()

### moyenne des transactions par clients

In [ ]:
round(df_total['client_id'].value_counts().mean(),2)

### évaluation de ces clients dit 'professionnels' et suppresion de la liste de clients pour la suite des analyses

In [ ]:
list_pro = ('c_1609','c_6714','c_3454','c_4958')

client_pro = df_total.loc[df_total['client_id'].isin(list_pro)] 


In [ ]:
df_total1 = df_total.drop(df_total.loc[df_total['client_id']=='c_1609'].index)

In [ ]:
df_total1 = df_total1.drop(df_total1.loc[df_total['client_id']=='c_6714'].index)


In [ ]:
df_total1 = df_total1.drop(df_total1.loc[df_total['client_id']=='c_3454'].index)


In [ ]:
df_total1 = df_total1.drop(df_total1.loc[df_total['client_id']=='c_4958'].index)

### groupage du dossier globale des ventes par mois 

In [ ]:
group_mois0 =df_total1.groupby(pd.Grouper(key='date', freq='1M')).sum()
group_mois0

## zoom sur le mois d'octobre pour identifier le problème et voir la marche à suivre

### on constate un probleme de CA sur octobre 2021 donc on va voir ceci sur un graphique

In [ ]:
ca_jour0= df_total1.groupby(['date']).sum()
loc_ca_jour_octobre = ca_jour0.loc['2021-10-01':'2021-10-31']
plt.figure(figsize=(12,8))
x= loc_ca_jour_octobre.index
y= loc_ca_jour_octobre['price']
plt.plot(x,y,label='CA octobre 2021')
plt.xlabel('date',size=14)
plt.ylabel("chiffre d'affaires ",size=14)
plt.xticks(rotation= 45)
plt.grid()
plt.legend()
plt.title("Courbe du chiffre d'affaires octobre 2021",size=18)
plt.show()            

## au vue de la courbe on constate un problème de chiffre d'affaires sur une majorité du mois d'octobre 2021

In [ ]:
dfoctobre =df_total1.set_index(df_total1['date'])
vente_octobre02_27= dfoctobre.loc['2021-10-02':'2021-10-27']
plt.figure(figsize = (6,6))
vente_octobre02_27['categ'].value_counts(normalize=True).plot(kind='pie', autopct='%1.0f%%')
plt.ylabel("")
plt.title("Répartition CA selon catégories - Octobre entre le 02 et le 27 ",size=18)
plt.ylabel("Chiffre d'affaire",size=14)
plt.legend()
plt.show()


# suppression du mois d'octobre en totalité

In [ ]:
exept_oct = (df_total1['date'] <'2021-10-01' ) | (df_total1['date'] > '2021-10-31')
df4 = df_total1.loc[exept_oct]
df4.head()

# 2ème PARTIE: CALCULS ET ANLAYSES DES DONNEES

## 2-1:QUESTIONS D'ANTOINE

### calcul du chiffre d'affaires par jour puis par mois et par catégories

In [ ]:
ca_jour= df4.groupby(['date']).sum()

In [ ]:
ca_mois_categ = df4.groupby([pd.Grouper(key='date', freq='1M'),'categ']).sum()
ca_mois_categ = ca_mois_categ.reset_index('categ')

In [ ]:
cat0_mois = ca_mois_categ.loc[ca_mois_categ['categ']==0.0]
cat1_mois =ca_mois_categ.loc[ca_mois_categ['categ']==1.0]
cat2_mois = ca_mois_categ.loc[ca_mois_categ['categ']==2.0]

In [ ]:
CA_jour_moy = round(ca_jour['price'].mean(),2)
print('le chiffre  d_affaires journalier moyen par jour est de',CA_jour_moy,'euros')

In [ ]:
plt.figure(figsize=(18,10))
x= ca_jour.index
y= ca_jour['price']
plt.plot(x,y,label='Chiffre d_affaires jour')
plt.xlabel('date',size=14)
plt.ylabel('chiffre d_affaires journalier',size=14)
plt.grid()
plt.legend()
plt.title("évolution du chiffre d'affaires journalier",size=20)
plt.show()            

In [ ]:
group_mois =df4.groupby(pd.Grouper(key='date', freq='1M')).sum()
group_mois= group_mois.drop('2021-10-31')

### moyenne CA mensuel

In [ ]:
CA_mois_moy = round(group_mois['price'].mean(),2)
print('le chiffre d_affaires mensuel moyen est de',CA_mois_moy,'euros')

###  création de la colonne moyenne mobile 

In [ ]:
group_mois['moyenne mobile']= group_mois.price.rolling(5).mean()


### courbes du chiffre d'affaire total et de la moyenne mobile

In [ ]:
plt.figure(figsize=(16,10))
x= group_mois.index
y= group_mois['price']
y1= group_mois['moyenne mobile']
plt.plot(x,y1,color='green',label='moyenne mobile')
plt.plot(x,y,label='CA mensuel')
plt.scatter(x,y,color='red')
plt.xlabel('date',size=14)
plt.ylabel("chiffre d'affaires mensuel",size=14)
plt.xticks(rotation= 45)
plt.grid()
plt.legend()
plt.title("Courbe de la moyenne mobile et du chiffre d'affaires mensuel",size=18)
plt.show()            

### courbes des chiffres d'affaires par catégories et leur moyenne mobile respective

In [ ]:
plt.figure(figsize=(16,10))
x= cat0_mois.index
y= cat0_mois['price']
y3 = cat0_mois.price.rolling(5).mean()
y4 =cat1_mois.price.rolling(5).mean()
y5=cat2_mois.price.rolling(5).mean()
x1=cat1_mois.index
y1= cat1_mois['price']
x2=cat2_mois.index
y2= cat2_mois['price']
plt.plot(x,y,label='CA catégorie 0')
plt.plot(x,y1,color='green',label='CA catégorie 1')
plt.plot(x2,y2,color='red',label='CA catégorie 2')
plt.plot(x,y3, label='moyenne mobile catégorie 0', color='blue',linestyle = '--' )
plt.plot(x,y4, label='moyenne mobile catégorie 1', color='green',linestyle = '--' )
plt.plot(x,y5, label='moyenne mobile catégorie 2', color='red',linestyle = '--' )
plt.xlabel('date',size=14)
plt.ylabel("chiffre d'affaires mensuel",size=14)
plt.xticks(rotation= 45)
plt.grid()
plt.legend()
plt.title("Courbes des chiffres d'affaires par catégorie et leur moyenne mobile",size=18)
plt.show()            

In [ ]:
plt.figure(figsize=(16,10))
x= cat0_mois.index
y= cat0_mois['price']
y3 = cat0_mois.price.rolling(5).mean()
y4 =cat1_mois.price.rolling(5).mean()
y5=cat2_mois.price.rolling(5).mean()
x1=cat1_mois.index
y1= cat1_mois['price']
x2=cat2_mois.index
y2= cat2_mois['price']

plt.plot(x,y3, label='moyenne mobile catégorie 0', color='blue',linestyle = '--' )
plt.plot(x,y4, label='moyenne mobile catégorie 1', color='green',linestyle = '--' )
plt.plot(x,y5, label='moyenne mobile catégorie 2', color='red',linestyle = '--' )
plt.xlabel('date',size=14)
plt.ylabel("chiffre d'affaires mensuel",size=14)
plt.xticks(rotation= 45)
plt.grid()
plt.legend()
plt.title("Courbes des moyennes mobiles des chiffres d'affaires de chaque catégorie",size=18)
plt.show()            

### chiffre d'affaires total avant suppression des  clients pro

In [ ]:
CA_total = round(transac_prod['price'].sum(),2)
print('le chiffre d_affaires total est de',CA_total,'euros')

### chiffre d'affaires sans les clients dit professionnels

In [ ]:
CA_total2 = round(df_total1.price.sum(),2)
print('le chiffre d_affaires des clients dit professionnels est de',client_pro.price.sum(),'euros')
print('le chiffre d_affaires pro représente',round((client_pro.price.sum())/(df_total.price.sum())*100,2),
      '% du chiffre d_affaires total')
print('le chiffre d_affaires sans les clients dit professionnels est de',CA_total2,'euros')

### meilleures ventes en fonction du chiffre d'affaires

In [ ]:
df_id_prod = transac_prod.groupby('id_prod').sum()

In [ ]:
top_prod= df_id_prod.sort_values(by='price',ascending=False).head(5)
top_prod


In [ ]:
plt.figure(figsize=(10,6))
x=top_prod.index
y=top_prod['price']
plt.bar(x,y)
plt.title("Meilleures ventes en fonction du chiffre d'affaires",size=18)
plt.ylabel("chiffre d'affaires",size=14)
plt.xlabel('identifiant produit',size=14)
plt.show()

### meilleures ventes en fonction du nombre de vente

In [ ]:
df_total['id_prod'].value_counts().head()

### moins bonnes ventes en fonction du chiffres d'affaires

In [ ]:
flop_prod= df_id_prod.sort_values(by='price',ascending=True).head()
flop_prod

In [ ]:
plt.figure(figsize=(10,6))
x=flop_prod.index
y=flop_prod['price']
plt.bar(x,y)
plt.title("Moins bonnes ventes en fonction du chiffre d'affaires",size=18)
plt.ylabel("chiffre d'affaires",size=14)
plt.xlabel('identifiant produit',size=14)
plt.show()

### moins bonnes ventes en fonction du nombre de vente

In [ ]:
df_total['id_prod'].value_counts().tail()

## J'ai effectué cette double analyse des tops et des flops car Annabelle dans son mail nous indique vouloir ces analyses pour faire évoluer sa stratégie marketing et peut être faire évoluer ses prix donc ici peut être faut il faire évoluer les prix des produits les plus vendus pour qu'ils deviennent ceux dont le CA est le meilleur

### calcul et visualisation du chiffre d'affaires par âge

In [ ]:
group_age_som = round(df_total.groupby('age').sum(),2)
top_age = group_age_som.sort_values(by='price',ascending=False).head()
top_age['price']

In [ ]:
plt.figure(figsize=(12,8))
x=group_age_som.index
y=group_age_som['price']
plt.bar(x,y)
plt.title("Chiffre d'affaires en fonction de l'age",size=18)
plt.ylabel("Chiffre d'affaires",size=14)
plt.xlabel('Age',size=14)
plt.show()

In [ ]:
print("l'âge moyen des clients est de",round(df4['age'].mean(),2),'ans')

In [ ]:
print("l'âge median des clients est de",round(df4['age'].median(),2),'ans')

# Les ages de 43 et 19 ans représentent les meilleurs chiffres d'affaires

### calcul et visualisation du chiffre d'affaires par tranche d'âge

In [ ]:
group_tranch_age_som = round(df4.groupby('tranche age').sum(),2)
plt.figure(figsize=(12,6))
x=group_tranch_age_som.index
y=group_tranch_age_som['price']
plt.bar(x,y)
plt.title("Chiffre d'affaires en fonction des tranches d'age",size=18)
plt.ylabel("Chiffre d'affaires",size=14)
plt.xlabel("Tranche d'age",size=14)
plt.show()

# la globalité du CA se regroupe sur 4 tranches alant de 21 à 60ans

### répartition des ventes par sexe

In [ ]:
ca_sex= df4.groupby(["sex"]).sum()
femme = round(ca_sex.loc['f','price']*100/CA_total2,1)
homme =round(ca_sex.loc['m','price']*100/CA_total2,1)

In [ ]:
plt.figure(figsize = (6,6))
plt.pie(labels = ['homme','femme' ],
        x=[homme,femme],
        autopct='%.2f%%')
plt.legend()
plt.title('répartition des ventes par sexe')
plt.show()

### calcul du panier moyen

In [ ]:
pan_moy=df4.groupby(['session_id']).sum()
print('le panier moyen est de',round(pan_moy['price'].mean(),2),'euros')
print('le panier median est de',round(pan_moy['price'].median(),2),'euros')

### graphique de la répartition des ventes par catégories

In [ ]:
df_categ = df4.groupby('categ').sum()
cat0 =round(df_categ.loc[0.0,'price']*100/CA_total2,2)
cat1 =round(df_categ.loc[1.0,'price']*100/CA_total2,2)
cat2 =round(df_categ.loc[2.0,'price']*100/CA_total2,2)

In [ ]:
plt.figure(figsize = (6,6))
plt.pie(labels = ['catégorie 0','catégorie 1','catégorie 2' ],
        x=[cat0,cat1,cat2],
        autopct='%.2f%%',colors=['blue','green','red'])
plt.title('répartition des ventes par catégories')
plt.legend(bbox_to_anchor=(1.2, 1) )
plt.show()

### graphique de répartition des ventes par catégories et par sexe

In [ ]:
df_categ_sex = df4.groupby(['categ','sex']).sum()
df_categ_sex1 =df_categ_sex.reset_index('sex')
cat0_fem =round(df_categ_sex.iloc[0,0]*100/CA_total2,2)
cat1_fem =round(df_categ_sex.iloc[2,0]*100/CA_total2,2)
cat2_fem =round(df_categ_sex.iloc[4,0]*100/CA_total2,2)
cat0_hom =round(df_categ_sex.iloc[1,0]*100/CA_total2,2)
cat1_hom =round(df_categ_sex.iloc[3,0]*100/CA_total2,2)
cat2_hom =round(df_categ_sex.iloc[5,0]*100/CA_total2,2)

In [ ]:
df4.groupby(['categ','sex']).sum()

In [ ]:
plt.figure(figsize = (6,6))
x1=[cat0_hom,cat1_hom,cat2_hom]
plt.pie(x1,labels = ['catégorie 0 homme','catégorie 1 homme','catégorie 2 homme'], autopct='%.2f%%',
        colors=['moccasin','yellow','orange'])
plt.title('répartition des ventes par catégorie pour les hommes',size=18)
plt.show()

plt.figure(figsize = (6,6))
x=[cat0_fem,cat1_fem,cat2_fem]
plt.pie(x,labels = ['catégorie 0 femme','catégorie 1 femme','catégorie 2 femme'],autopct='%.2f%%',
        colors=['lightblue','lightskyblue','royalblue'])
plt.title('répartition des ventes par catégorie pour les femmes',size=18)
plt.show()

### graphique et calcul de la fréquence d'achats par clients

In [ ]:
round(df4['client_id'].value_counts().mean(),2)

In [ ]:
round(df4['client_id'].value_counts().median(),2)

In [ ]:
plt.figure(figsize = (10,8))
plt.hist(df4['client_id'].value_counts(),bins=40)
plt.title('Fréquence achat des clients',size=18)
plt.xlabel('Fréquence',size=14)
plt.ylabel('Nombre de clients',size=14)
plt.show()

In [ ]:
fid_client = df4.groupby('client_id').count().reset_index()
fid_client['achat'] = fid_client.session_id
fid_client = fid_client[['client_id', 'achat']]
len(fid_client)

In [ ]:
round(fid_client['achat'].mean(),2)

In [ ]:
propo_client_moins = round(fid_client[fid_client['achat'] == 1].shape[0]*100/len(fid_client),2)
print("la proportion de clients ne faisant qu'une commande est de",propo_client_moins,'%')

In [ ]:
propo_client_plus = round(fid_client[fid_client['achat'] >= 10].shape[0]*100/len(fid_client),2)
print("la proportion de clients faisant au moins 10 commandes est de",propo_client_plus,'%')

### graphique de la distribution des ages des clients

In [ ]:
plt.figure(figsize = (10,8))
plt.hist(customer['tranche age'].sort_values(),bins=50)
plt.title("Distribution des clients par tranche d'ages",size=18)
plt.xlabel('Age',size=14)
plt.ylabel('Nombre de clients',size=14)
plt.show()

### graphique du panier moyen en fonction du sexe

In [ ]:
session_sex = df4[["session_id","sex"]]
panier_sex = pd.merge(session_sex,pan_moy, left_on="session_id", right_on="session_id")
plt.figure(figsize=(10,8))
sns.boxplot( y=panier_sex["sex"], x=panier_sex["price"] )
plt.title('panier moyen par sexe',size=18)
plt.show()

### courbe de lorenz et coefficiant de Gini sans les 4 clients dit 'professionnels' ni le mois d'octobre 2021

In [ ]:
price = df4['price'].values
lorenz_price = np.cumsum(np.sort(price)) / price.sum()
#Tri des individus dans l'ordre croissant des valeurs de la variable, 
#Calcul de la somme cumulée et normalisation en divisant par la somme des observations
plt.figure(figsize = (8,6))
plt.plot(np.linspace(0,1,len(lorenz_price)), lorenz_price, drawstyle='steps-post', color='rosybrown', label='Lorenz')
plt.fill_between(np.linspace(0,1,len(lorenz_price)) ,lorenz_price , color='#539ecd')
plt.plot([0, 1], [0, 1], 'r-', lw=2, label='Distribution égalitaire')

plt.title('Courbe de Lorenz des prix de vente')
plt.xlabel("Distribution des ventes")
plt.ylabel("Cumul des prix de ventes")
plt.show()

In [ ]:
aire_sous_lorenz = lorenz_price[:-1].sum()/len(lorenz_price) 
S = 0.5 - aire_sous_lorenz
gini = round(2*S,3)
gini

### courbe de lorenz et coefficiant de Gini avec les 4 clients dit 'professionnels' et avec le mois d'octobre 2021

In [ ]:
price1 = df_total['price'].values
lorenz_price1 = np.cumsum(np.sort(price1)) / price1.sum()
#Tri des individus dans l'ordre croissant des valeurs de la variable, 
#Calcul de la somme cumulée et normalisation en divisant par la somme des observations
plt.figure(figsize = (8,6))
plt.plot(np.linspace(0,1,len(lorenz_price1)), lorenz_price1, drawstyle='steps-post', color='rosybrown', label='Lorenz')
plt.fill_between(np.linspace(0,1,len(lorenz_price1)) ,lorenz_price1 , color='#539ecd')
plt.plot([0, 1], [0, 1], 'r-', lw=2, label='Distribution égalitaire')

plt.title('Courbe de Lorenz avec les 4 clients pro et octobre')
plt.xlabel("Distribution des ventes ")
plt.ylabel("Cumul des prix de ventes ")
plt.show()

In [ ]:
aire_sous_lorenz1 = lorenz_price1[:-1].sum()/len(lorenz_price1) 
S = 0.5 - aire_sous_lorenz1
gini1 = round(2*S,3)
gini1

In [ ]:
plt.figure(figsize = (10,8))
plt.hist(product1['price'],bins=50)
plt.xlabel('prix',size=12)
plt.ylabel('féquence',size=12)
plt.title('distibution des prix des livres',size=18)
plt.show()

### graphique des prix en fonction de la catégorie

In [ ]:
plt.figure(figsize=(10,8))
my_colors = ['blue','green','red']
sns.set_palette( my_colors )
sns.boxplot( x=product1['categ'], y=product1['price'])
plt.title('Prix selon catégorie')
plt.ylabel('Prix')
plt.xlabel('catégorie')
plt.show()

## 2-2: QUESTION DE JULIE

## graphique heatmap du lien entre sexe du client et les catégories de livres achetés

In [ ]:
df_sex_categ = pd.crosstab(df4['sex'],df4['categ'])

plt.figure(figsize=(10,8))
sns.heatmap(df_sex_categ,cmap = 'coolwarm')
plt.title('lien entre catégorie de livre et sexe du client',size=18)
plt.xlabel('catégorie',size=14)
plt.ylabel('sexe du client',size=14)
plt.show()

In [ ]:
pd.crosstab(df4['sex'],df4['categ'])

### test de normalité de Shapiro

In [ ]:
stat, p = shapiro(df_sex_categ)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### test d'indépendance du chi 2 sur variables quali/quali

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%
    
chi2, p, dof, ex = chi2_contingency(df_sex_categ, correction=False)

In [ ]:
p
alpha = 0.05
if p > alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)

## graphique de relation ente age et panier moyen

In [ ]:
session_age = df4[['session_id','age']]
panier_age = pd.merge(session_age,pan_moy, left_on="session_id", right_on="session_id")
plt.figure(figsize=(10,8))
sns.regplot(y=panier_age['price'], x=panier_age['age_x'],ci = None,line_kws = {'color': 'red'})
plt.title('Relation entre age et panier moyen',size=18)
plt.ylabel('Montant du panier',size=14)
plt.xlabel('Age',size=14)
plt.show()

### test de normalité de Shapiro 

In [ ]:
stat, p = shapiro(pan_moy)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### Test non paramétrique de corrélation de Spearman sur variables quanti/quanti

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%

coef, p = spearmanr(panier_age['price'],panier_age['age_x'])
print(coef,p)
print('coefficient de correlation Spearmans: %.3f' % coef)
alpha = 0.05
if p > alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)

## graphique de relation entre l'age des clients et les prix des produits achetés

In [ ]:
plt.figure(figsize=(10,8))
sns.regplot(y=df4['price'], x=df4['age'],ci = None,line_kws = {'color': 'red'})
plt.title('Relation entre age et prix produits achetés',size=18)
plt.ylabel('Prix',size=14)
plt.xlabel('Age',size=14)
plt.show()

### test de normalité de Shapiro

In [ ]:
stat, ps = shapiro(df4['age'])
print('Statistics=%.3f, p=%.3f' % (stat, ps))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### Test non paramétrique de corrélation de Spearman sur variables quanti/quanti

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%

coef, p = spearmanr(df4['price'],df4['age'])
print(coef,p)
print('coefficient de correlation Spearmans: %.3f' % coef)
alpha = 0.05
if p > alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)

## graphique de relation entre l'age des clients et la catégorie de livres achetés

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot( x=df4['categ'], y=df4['age'] )
plt.title('Age selon catégorie')
plt.ylabel('Age')
plt.xlabel('Catégorie')
plt.show()

### test de normalité de Shapiro

In [ ]:
stat, ps = shapiro(df4['age'])
print('Statistics=%.3f, p=%.3f' % (stat, ps))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### test de Kruskal-Wallis sur variables quali/quanti

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%
    
result =stats.kruskal(df4['categ'], df4['age'])
print(p)
alpha = 0.05
if p> alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)

In [ ]:
stats.ttest_ind(df4['categ'], df4['age'])
p

## graphique de relation entre l'age des clients et la fréquence d'achat

In [ ]:
age=customer[["client_id","age"]]
freq = pd.crosstab(index = df4["client_id"], columns='total')
freq = pd.merge(freq,age,left_on="client_id", right_on="client_id")
plt.figure(figsize=(10,8))
sns.regplot(y=freq['total'], x=freq['age'],ci = None,line_kws = {"color": "red"})
plt.title("Relation entre age et fréquence d'achat",size=18)
plt.ylabel("Fréquence",size=14)
plt.xlabel("Age",size=14)
plt.show()

### test de normalité de Shapiro

In [ ]:
stat, ps = shapiro(freq['age'])
print('Statistics=%.3f, p=%.3f' % (stat, ps))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### Test non paramétrique de corrélation de Spearman sur variables quanti/quanti

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%

coef, p = spearmanr(freq['total'],freq['age'])
print(coef,p)
print('coefficiant de correlation Spearmans: %.3f' % coef)
alpha = 0.05
if p > alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)

## graphique de relation entre l'age des clients et le chiffre d'affaires

In [ ]:
plt.figure(figsize=(10,8))
ca_age=df4[["client_id","price"]]
ca_age=ca_age.groupby(["client_id"]).sum()
ca_age = ca_age.reset_index()
age=customer[["client_id","age"]]
ca_age=pd.merge(ca_age,age,left_on="client_id", right_on="client_id")
sns.regplot(y=ca_age['price'], x=ca_age['age'],ci = None,line_kws = {"color": "red"})
plt.title("Relation entre age et Chiffre d'affaires",size=18)
plt.ylabel("Chiffre d'affaires",size=14)
plt.xlabel('Age',size=14)
plt.show()

### test de normalité de Shapiro

In [ ]:
stat, ps = shapiro(ca_age['age'])
print('Statistics=%.3f, p=%.3f' % (stat, ps))
alpha = 0.05
if p < alpha:
    print('la distribution des données ne suit pas la loi normale p=%.3f' % p)
else:
    print('la distribution des données suit la loi normale p=%.3f' % p)

### Test non paramétrique de corrélation de Spearmansur variables quanti/quanti

In [ ]:
### On pose les hypothèses de départ :
# H0 : Variables indépendantes si p-value > 5%
# H1 : Variables non indépendantes si p-value < 5%

coef, p = spearmanr(ca_age['price'],ca_age['age'])
print(coef,p)
print('coefficiant de correlation Spearmans: %.3f' % coef)
alpha = 0.05
if p > alpha:
	print('les echantillons ne sont pas corrélés (ne pas rejeter H0) p=%.3f' % p)
else:
	print('les echantillons sont corrélés (rejeter H0) p=%.3f' % p)